In [66]:
#All needed libaries for the project
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings
warnings.filterwarnings('ignore')

In [67]:
#Import data from the web
data = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
data.head()

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [68]:
data['Symbol'] = data['Symbol'].str.replace('.', '-')
symbol_list = data['Symbol'].unique().tolist()
symbol_list

['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ACN',
 'ADBE',
 'AMD',
 'AES',
 'AFL',
 'A',
 'APD',
 'ABNB',
 'AKAM',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'AON',
 'APA',
 'AAPL',
 'AMAT',
 'APTV',
 'ACGL',
 'ADM',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'AXON',
 'BKR',
 'BALL',
 'BAC',
 'BK',
 'BBWI',
 'BAX',
 'BDX',
 'BRK-B',
 'BBY',
 'BIO',
 'TECH',
 'BIIB',
 'BLK',
 'BX',
 'BA',
 'BKNG',
 'BWA',
 'BXP',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BF-B',
 'BLDR',
 'BG',
 'CDNS',
 'CZR',
 'CPT',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CTLT',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'COR',
 'CNC',
 'CNP',
 'CF',
 'CHRW',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMCSA',
 'CMA'

In [69]:
#Adjust end_date to the current needs
end_date = '2023-09-27'
start_date = pd.to_datetime(end_date)-pd.DateOffset(365*8)
start_date

Timestamp('2015-09-29 00:00:00')

In [70]:
df = yf.download(tickers=symbol_list, start = start_date, end = end_date).stack()
df.index.names = ['date', 'ticker']
df.columns = df.columns.str.lower()
df

[*********************100%***********************]  503 of 503 completed


1 Failed download:
['VLTO']: Exception("%ticker%: Data doesn't exist for startDate = 1443499200, endDate = 1695787200")


adj close       close        high         low        open  \
date       ticker                                                               
2015-09-29 A        31.534700   33.740002   34.060001   33.240002   33.360001   
           AAL      37.361622   39.180000   39.770000   38.790001   39.049999   
           AAPL     24.684566   27.264999   28.377501   26.965000   28.207500   
           ABBV     36.674236   52.790001   54.189999   51.880001   53.099998   
           ABT      33.644741   39.500000   40.150002   39.029999   39.259998   
...                       ...         ...         ...         ...         ...   
2023-09-26 XYL      88.968475   89.519997   90.849998   89.500000   90.379997   
           YUM     122.811577  124.010002  124.739998  123.449997  124.239998   
           ZBH     111.992920  112.459999  117.110001  112.419998  116.769997   
           ZBRA    223.960007  223.960007  226.649994  222.580002  225.970001   
           ZTS     176.048233  176.869995  178.449997  176.270004  176.580002   

                        volume  
date       ticker               
2015-09-29 A         2252400.0  
           AAL       7478800.0  
           AAPL    293461600.0  
           ABBV     12842800.0  
           ABT      12287500.0  
...                        ...  
2023-09-26 XYL       1322400.0  
           YUM       1500600.0  
           ZBH       3610500.0  
           ZBRA       355400.0  
           ZTS       1463200.0  

[993179 rows x 6 columns]

***Garman Klass Volatility***
is a tool developed by Mark Garman and Steven W. Klass. The model is a method for estimating the volatility of financial instruments, like stocks or commodities. The Garman-Klass Volatility formula is recognized for its efficiency in capturing price range and movements within the trading day, making it valuable for financial analysts and traders who require a more nuanced understanding of market dynamics. The result it dependent from the person, technicly lower result will idicate more calm market. The higher the Garman-Klass volatility, the higher the risk associated with the asset, as it suggests greater price movement.

In [71]:
df['garman'] = ((np.log(df['high'])- np.log(df['low']))**2)/2-(2*np.log(2)-1)*((np.log(df['adj close']) -np.log(df['open']))**2)
df

adj close       close        high         low        open  \
date       ticker                                                               
2015-09-29 A        31.534700   33.740002   34.060001   33.240002   33.360001   
           AAL      37.361622   39.180000   39.770000   38.790001   39.049999   
           AAPL     24.684566   27.264999   28.377501   26.965000   28.207500   
           ABBV     36.674236   52.790001   54.189999   51.880001   53.099998   
           ABT      33.644741   39.500000   40.150002   39.029999   39.259998   
...                       ...         ...         ...         ...         ...   
2023-09-26 XYL      88.968475   89.519997   90.849998   89.500000   90.379997   
           YUM     122.811577  124.010002  124.739998  123.449997  124.239998   
           ZBH     111.992920  112.459999  117.110001  112.419998  116.769997   
           ZBRA    223.960007  223.960007  226.649994  222.580002  225.970001   
           ZTS     176.048233  176.869995  178.449997  176.270004  176.580002   

                        volume    garman  
date       ticker                         
2015-09-29 A         2252400.0 -0.000926  
           AAL       7478800.0 -0.000443  
           AAPL    293461600.0 -0.005572  
           ABBV     12842800.0 -0.051964  
           ABT      12287500.0 -0.008803  
...                        ...       ...  
2023-09-26 XYL       1322400.0  0.000016  
           YUM       1500600.0  0.000002  
           ZBH       3610500.0  0.000161  
           ZBRA       355400.0  0.000133  
           ZTS       1463200.0  0.000072  

[993179 rows x 7 columns]

***The Relative Strength Index (RSI)*** is a momentum oscillator used in technical analysis that measures the speed and change of price movements of a security. Below 30: The asset is considered oversold. This suggests that the asset may be undervalued or that a reversal upward could occur soon. It's typically seen as a buying opportunity by traders.
Above 70: The asset is considered overbought. This suggests that the asset may be overvalued or that a reversal downward could happen soon. Traders might see this as a signal to sell.
Around 50: An RSI value near 50 is considered neutral, indicating no strong momentum in either direction for the asset's price.

In [72]:
df['rsi'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close = x, length=20))
df

adj close       close        high         low        open  \
date       ticker                                                               
2015-09-29 A        31.534700   33.740002   34.060001   33.240002   33.360001   
           AAL      37.361622   39.180000   39.770000   38.790001   39.049999   
           AAPL     24.684566   27.264999   28.377501   26.965000   28.207500   
           ABBV     36.674236   52.790001   54.189999   51.880001   53.099998   
           ABT      33.644741   39.500000   40.150002   39.029999   39.259998   
...                       ...         ...         ...         ...         ...   
2023-09-26 XYL      88.968475   89.519997   90.849998   89.500000   90.379997   
           YUM     122.811577  124.010002  124.739998  123.449997  124.239998   
           ZBH     111.992920  112.459999  117.110001  112.419998  116.769997   
           ZBRA    223.960007  223.960007  226.649994  222.580002  225.970001   
           ZTS     176.048233  176.869995  178.449997  176.270004  176.580002   

                        volume    garman        rsi  
date       ticker                                    
2015-09-29 A         2252400.0 -0.000926        NaN  
           AAL       7478800.0 -0.000443        NaN  
           AAPL    293461600.0 -0.005572        NaN  
           ABBV     12842800.0 -0.051964        NaN  
           ABT      12287500.0 -0.008803        NaN  
...                        ...       ...        ...  
2023-09-26 XYL       1322400.0  0.000016  26.146751  
           YUM       1500600.0  0.000002  36.057180  
           ZBH       3610500.0  0.000161  31.893251  
           ZBRA       355400.0  0.000133  29.494977  
           ZTS       1463200.0  0.000072  42.623454  

[993179 rows x 8 columns]

Bollinger Bands are used not only to identify potential overbought or oversold conditions but also to catch breakouts and determine entry and exit points. However, like all trading indicators, Bollinger Bands should be used in conjunction with other analysis tools and within a well-thought-out trading strategy to minimize risks.

In [73]:
df['bb_low'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length = 20).iloc[:,0])
df['bb_mid'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length = 20).iloc[:,1])
df['bb_high'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length = 20).iloc[:,2])
df

adj close       close        high         low        open  \
date       ticker                                                               
2015-09-29 A        31.534700   33.740002   34.060001   33.240002   33.360001   
           AAL      37.361622   39.180000   39.770000   38.790001   39.049999   
           AAPL     24.684566   27.264999   28.377501   26.965000   28.207500   
           ABBV     36.674236   52.790001   54.189999   51.880001   53.099998   
           ABT      33.644741   39.500000   40.150002   39.029999   39.259998   
...                       ...         ...         ...         ...         ...   
2023-09-26 XYL      88.968475   89.519997   90.849998   89.500000   90.379997   
           YUM     122.811577  124.010002  124.739998  123.449997  124.239998   
           ZBH     111.992920  112.459999  117.110001  112.419998  116.769997   
           ZBRA    223.960007  223.960007  226.649994  222.580002  225.970001   
           ZTS     176.048233  176.869995  178.449997  176.270004  176.580002   

                        volume    garman        rsi    bb_low    bb_mid  \
date       ticker                                                         
2015-09-29 A         2252400.0 -0.000926        NaN       NaN       NaN   
           AAL       7478800.0 -0.000443        NaN       NaN       NaN   
           AAPL    293461600.0 -0.005572        NaN       NaN       NaN   
           ABBV     12842800.0 -0.051964        NaN       NaN       NaN   
           ABT      12287500.0 -0.008803        NaN       NaN       NaN   
...                        ...       ...        ...       ...       ...   
2023-09-26 XYL       1322400.0  0.000016  26.146751  4.488345  4.570270   
           YUM       1500600.0  0.000002  36.057180  4.816569  4.846536   
           ZBH       3610500.0  0.000161  31.893251  4.749948  4.789615   
           ZBRA       355400.0  0.000133  29.494977  5.400991  5.539167   
           ZTS       1463200.0  0.000072  42.623454  5.158939  5.217753   

                    bb_high  
date       ticker            
2015-09-29 A            NaN  
           AAL          NaN  
           AAPL         NaN  
           ABBV         NaN  
           ABT          NaN  
...                     ...  
2023-09-26 XYL     4.652196  
           YUM     4.876502  
           ZBH     4.829283  
           ZBRA    5.677342  
           ZTS     5.276568  

[993179 rows x 11 columns]

***The Average True Range (ATR)*** is a technical analysis indicator that measures market volatility by decomposing the entire range of an asset price for that period. It provides a measure of price volatility.
Higher ATR: A high ATR value indicates high volatility, meaning the price is moving significantly within a given period. This can be seen in markets with large price fluctuations or around major news events.
Lower ATR: A low ATR value suggests lower volatility, with less price movement and potentially a more stable market condition.

In [74]:
def compute_atr (data):
    atr = pandas_ta.atr(high=data['high'], low=data['low'], close=data['close'], length=14)
    return atr.sub(atr.mean()).div(atr.std())
df['atr'] = df.groupby(level=1, group_keys=False).apply(compute_atr)
df

adj close       close        high         low        open  \
date       ticker                                                               
2015-09-29 A        31.534700   33.740002   34.060001   33.240002   33.360001   
           AAL      37.361622   39.180000   39.770000   38.790001   39.049999   
           AAPL     24.684566   27.264999   28.377501   26.965000   28.207500   
           ABBV     36.674236   52.790001   54.189999   51.880001   53.099998   
           ABT      33.644741   39.500000   40.150002   39.029999   39.259998   
...                       ...         ...         ...         ...         ...   
2023-09-26 XYL      88.968475   89.519997   90.849998   89.500000   90.379997   
           YUM     122.811577  124.010002  124.739998  123.449997  124.239998   
           ZBH     111.992920  112.459999  117.110001  112.419998  116.769997   
           ZBRA    223.960007  223.960007  226.649994  222.580002  225.970001   
           ZTS     176.048233  176.869995  178.449997  176.270004  176.580002   

                        volume    garman        rsi    bb_low    bb_mid  \
date       ticker                                                         
2015-09-29 A         2252400.0 -0.000926        NaN       NaN       NaN   
           AAL       7478800.0 -0.000443        NaN       NaN       NaN   
           AAPL    293461600.0 -0.005572        NaN       NaN       NaN   
           ABBV     12842800.0 -0.051964        NaN       NaN       NaN   
           ABT      12287500.0 -0.008803        NaN       NaN       NaN   
...                        ...       ...        ...       ...       ...   
2023-09-26 XYL       1322400.0  0.000016  26.146751  4.488345  4.570270   
           YUM       1500600.0  0.000002  36.057180  4.816569  4.846536   
           ZBH       3610500.0  0.000161  31.893251  4.749948  4.789615   
           ZBRA       355400.0  0.000133  29.494977  5.400991  5.539167   
           ZTS       1463200.0  0.000072  42.623454  5.158939  5.217753   

                    bb_high       atr  
date       ticker                      
2015-09-29 A            NaN       NaN  
           AAL          NaN       NaN  
           AAPL         NaN       NaN  
           ABBV         NaN       NaN  
           ABT          NaN       NaN  
...                     ...       ...  
2023-09-26 XYL     4.652196  0.033800  
           YUM     4.876502  0.142547  
           ZBH     4.829283 -0.381708  
           ZBRA    5.677342 -0.057389  
           ZTS     5.276568  0.651515  

[993179 rows x 12 columns]

***The Moving Average Convergence Divergence (MACD)*** is a trend-following momentum indicator that shows the relationship between two moving averages of a security’s price. Negative MACD Value: A negative MACD value suggests that the 12-period EMA is below the 26-period EMA, indicating that short-term momentum is weaker than long-term momentum. This situation is typically interpreted as bearish and may suggest that it's not the optimal time to buy.

In [75]:
def compute_macd (close):
    macd = pandas_ta.macd(close=close, length=14).iloc[:,0]
    return macd.sub(macd.mean()).div(macd.std())
df['macd'] = df.groupby(level=1, group_keys=False)['adj close'].apply(compute_macd)
df

adj close       close        high         low        open  \
date       ticker                                                               
2015-09-29 A        31.534700   33.740002   34.060001   33.240002   33.360001   
           AAL      37.361622   39.180000   39.770000   38.790001   39.049999   
           AAPL     24.684566   27.264999   28.377501   26.965000   28.207500   
           ABBV     36.674236   52.790001   54.189999   51.880001   53.099998   
           ABT      33.644741   39.500000   40.150002   39.029999   39.259998   
...                       ...         ...         ...         ...         ...   
2023-09-26 XYL      88.968475   89.519997   90.849998   89.500000   90.379997   
           YUM     122.811577  124.010002  124.739998  123.449997  124.239998   
           ZBH     111.992920  112.459999  117.110001  112.419998  116.769997   
           ZBRA    223.960007  223.960007  226.649994  222.580002  225.970001   
           ZTS     176.048233  176.869995  178.449997  176.270004  176.580002   

                        volume    garman        rsi    bb_low    bb_mid  \
date       ticker                                                         
2015-09-29 A         2252400.0 -0.000926        NaN       NaN       NaN   
           AAL       7478800.0 -0.000443        NaN       NaN       NaN   
           AAPL    293461600.0 -0.005572        NaN       NaN       NaN   
           ABBV     12842800.0 -0.051964        NaN       NaN       NaN   
           ABT      12287500.0 -0.008803        NaN       NaN       NaN   
...                        ...       ...        ...       ...       ...   
2023-09-26 XYL       1322400.0  0.000016  26.146751  4.488345  4.570270   
           YUM       1500600.0  0.000002  36.057180  4.816569  4.846536   
           ZBH       3610500.0  0.000161  31.893251  4.749948  4.789615   
           ZBRA       355400.0  0.000133  29.494977  5.400991  5.539167   
           ZTS       1463200.0  0.000072  42.623454  5.158939  5.217753   

                    bb_high       atr      macd  
date       ticker                                
2015-09-29 A            NaN       NaN       NaN  
           AAL          NaN       NaN       NaN  
           AAPL         NaN       NaN       NaN  
           ABBV         NaN       NaN       NaN  
           ABT          NaN       NaN       NaN  
...                     ...       ...       ...  
2023-09-26 XYL     4.652196  0.033800 -2.159188  
           YUM     4.876502  0.142547 -1.363696  
           ZBH     4.829283 -0.381708 -0.881067  
           ZBRA    5.677342 -0.057389 -1.600791  
           ZTS     5.276568  0.651515 -1.188278  

[993179 rows x 13 columns]

In [76]:
df['dollar_volume'] = df['adj close']*df['volume']/1e6
df

adj close       close        high         low        open  \
date       ticker                                                               
2015-09-29 A        31.534700   33.740002   34.060001   33.240002   33.360001   
           AAL      37.361622   39.180000   39.770000   38.790001   39.049999   
           AAPL     24.684566   27.264999   28.377501   26.965000   28.207500   
           ABBV     36.674236   52.790001   54.189999   51.880001   53.099998   
           ABT      33.644741   39.500000   40.150002   39.029999   39.259998   
...                       ...         ...         ...         ...         ...   
2023-09-26 XYL      88.968475   89.519997   90.849998   89.500000   90.379997   
           YUM     122.811577  124.010002  124.739998  123.449997  124.239998   
           ZBH     111.992920  112.459999  117.110001  112.419998  116.769997   
           ZBRA    223.960007  223.960007  226.649994  222.580002  225.970001   
           ZTS     176.048233  176.869995  178.449997  176.270004  176.580002   

                        volume    garman        rsi    bb_low    bb_mid  \
date       ticker                                                         
2015-09-29 A         2252400.0 -0.000926        NaN       NaN       NaN   
           AAL       7478800.0 -0.000443        NaN       NaN       NaN   
           AAPL    293461600.0 -0.005572        NaN       NaN       NaN   
           ABBV     12842800.0 -0.051964        NaN       NaN       NaN   
           ABT      12287500.0 -0.008803        NaN       NaN       NaN   
...                        ...       ...        ...       ...       ...   
2023-09-26 XYL       1322400.0  0.000016  26.146751  4.488345  4.570270   
           YUM       1500600.0  0.000002  36.057180  4.816569  4.846536   
           ZBH       3610500.0  0.000161  31.893251  4.749948  4.789615   
           ZBRA       355400.0  0.000133  29.494977  5.400991  5.539167   
           ZTS       1463200.0  0.000072  42.623454  5.158939  5.217753   

                    bb_high       atr      macd  dollar_volume  
date       ticker                                               
2015-09-29 A            NaN       NaN       NaN      71.028759  
           AAL          NaN       NaN       NaN     279.420098  
           AAPL         NaN       NaN       NaN    7243.972380  
           ABBV         NaN       NaN       NaN     470.999882  
           ABT          NaN       NaN       NaN     413.409756  
...                     ...       ...       ...            ...  
2023-09-26 XYL     4.652196  0.033800 -2.159188     117.651912  
           YUM     4.876502  0.142547 -1.363696     184.291052  
           ZBH     4.829283 -0.381708 -0.881067     404.350437  
           ZBRA    5.677342 -0.057389 -1.600791      79.595386  
           ZTS     5.276568  0.651515 -1.188278     257.593775  

[993179 rows x 14 columns]

In [77]:
last_cols = [c for c in df.columns.unique(0) if c not in ['dollar_volume', 'volume', 'open', 'high', 'low', 'close']]
data = (pd.concat([df.unstack('ticker')['dollar_volume'].resample('M').mean().stack('ticker').to_frame('dollar_volume'),
                   df.unstack()[last_cols].resample('M').last().stack('ticker')],
                  axis=1)).dropna()

data

dollar_volume   adj close       atr   bb_high    bb_low  \
date       ticker                                                            
2015-11-30 A          136.213756   39.086586 -1.033887  3.692471  3.547568   
           AAL        287.915789   39.429928  0.190822  3.827635  3.672028   
           AAPL      4029.441744   26.890560 -0.967900  3.369611  3.282983   
           ABBV       340.716422   40.770763 -0.526809  3.832284  3.735768   
           ABT        212.708766   38.483471 -1.064842  3.704588  3.660875   
...                          ...         ...       ...       ...       ...   
2023-09-30 OTIS       154.983158   78.671936 -1.028320  4.464683  4.374104   
           ABNB      1633.500725  132.279999 -1.006939  5.024801  4.857047   
           CEG        196.971017  108.026909 -0.436215  4.734007  4.653660   
           GEHC       212.350630   66.153511 -0.893478  4.270855  4.155782   
           KVUE       677.910277   20.218124 -0.899746  3.174263  3.024299   

                     bb_mid    garman      macd        rsi  
date       ticker                                           
2015-11-30 A       3.620020 -0.001902  0.567158  73.421608  
           AAL     3.749832 -0.000966 -0.418772  40.718972  
           AAPL    3.326297 -0.003209 -0.142789  55.537342  
           ABBV    3.784026 -0.056745  0.145677  49.376860  
           ABT     3.682732 -0.010572  0.335557  56.962446  
...                     ...       ...       ...        ...  
2023-09-30 OTIS    4.419394 -0.000020 -1.534536  33.116222  
           ABNB    4.940924  0.000213 -0.037854  44.494127  
           CEG     4.693834  0.000171  0.366876  55.245464  
           GEHC    4.213319  0.000185 -1.116463  40.922327  
           KVUE    3.099281  0.000032 -1.435621  35.706337  

[46894 rows x 9 columns]

In [78]:
data['dollar_volume'] = (data.loc[:, 'dollar_volume'].unstack('ticker').rolling(5*12, min_periods=12).mean().stack())

data['dollar_vol_rank'] = (data.groupby('date')['dollar_volume'].rank(ascending=False))
data

dollar_volume   adj close       atr   bb_high    bb_low  \
date       ticker                                                            
2015-11-30 A                 NaN   39.086586 -1.033887  3.692471  3.547568   
           AAL               NaN   39.429928  0.190822  3.827635  3.672028   
           AAPL              NaN   26.890560 -0.967900  3.369611  3.282983   
           ABBV              NaN   40.770763 -0.526809  3.832284  3.735768   
           ABT               NaN   38.483471 -1.064842  3.704588  3.660875   
...                          ...         ...       ...       ...       ...   
2023-09-30 OTIS       171.241018   78.671936 -1.028320  4.464683  4.374104   
           ABNB       909.311388  132.279999 -1.006939  5.024801  4.857047   
           CEG        175.389874  108.026909 -0.436215  4.734007  4.653660   
           GEHC              NaN   66.153511 -0.893478  4.270855  4.155782   
           KVUE              NaN   20.218124 -0.899746  3.174263  3.024299   

                     bb_mid    garman      macd        rsi  dollar_vol_rank  
date       ticker                                                            
2015-11-30 A       3.620020 -0.001902  0.567158  73.421608              NaN  
           AAL     3.749832 -0.000966 -0.418772  40.718972              NaN  
           AAPL    3.326297 -0.003209 -0.142789  55.537342              NaN  
           ABBV    3.784026 -0.056745  0.145677  49.376860              NaN  
           ABT     3.682732 -0.010572  0.335557  56.962446              NaN  
...                     ...       ...       ...        ...              ...  
2023-09-30 OTIS    4.419394 -0.000020 -1.534536  33.116222            277.0  
           ABNB    4.940924  0.000213 -0.037854  44.494127             37.0  
           CEG     4.693834  0.000171  0.366876  55.245464            268.0  
           GEHC    4.213319  0.000185 -1.116463  40.922327              NaN  
           KVUE    3.099281  0.000032 -1.435621  35.706337              NaN  

[46894 rows x 10 columns]

In [79]:
#Most liquidate stocks for each month.
data = data[data['dollar_vol_rank']<150].drop(['dollar_volume', 'dollar_vol_rank'], axis=1)
data

adj close       atr   bb_high    bb_low    bb_mid  \
date       ticker                                                       
2016-10-31 AAL      39.134331  0.402199  3.706314  3.604673  3.655493   
           AAPL     26.248026 -1.038688  3.353305  3.295542  3.324423   
           ABBV     40.620972 -0.893132  3.870876  3.762524  3.816700   
           ABT      34.463535 -1.035224  3.660400  3.559438  3.609919   
           ACN     103.970650 -0.996806  4.664445  4.641169  4.652807   
...                       ...       ...       ...       ...       ...   
2023-09-30 WMT      53.783062 -0.196381  4.023309  3.988367  4.005838   
           XOM     114.292953  0.601335  4.775304  4.695106  4.735205   
           MRNA     98.120003 -0.529511  4.788149  4.582514  4.685332   
           UBER     44.270000 -0.746098  3.917801  3.806654  3.862227   
           ABNB    132.279999 -1.006939  5.024801  4.857047  4.940924   

                     garman      macd        rsi  
date       ticker                                 
2016-10-31 AAL    -0.000176  1.131596  62.203534  
           AAPL   -0.002384 -0.195978  49.891058  
           ABBV   -0.044217 -0.760594  27.477815  
           ABT    -0.006968 -0.650888  38.008862  
           ACN    -0.004320 -0.135457  53.823656  
...                     ...       ...        ...  
2023-09-30 WMT    -0.000011  0.399459  54.722559  
           XOM     0.000026  1.400623  59.440178  
           MRNA    0.000146 -0.376899  38.747314  
           UBER    0.000441 -0.133973  45.005268  
           ABNB    0.000213 -0.037854  44.494127  

[12516 rows x 8 columns]

Calcualte monthly returns for different time horizons

In [80]:
def calculate_returns(data):
    outline_cutoff = 0.005
    lags=[1,2,3,6,9,12]
    for lag in lags:
        data[f'return_{lag}m'] = (data['adj close'].pct_change(lag).pipe(lambda x: x.clip(lower=x.quantile(outline_cutoff), upper=x.quantile(1-outline_cutoff))).add(1).pow(1/lag).sub(1))
    return data
data = data.groupby(level=1, group_keys=False).apply(calculate_returns).dropna()
data


adj close       atr   bb_high    bb_low    bb_mid  \
date       ticker                                                       
2017-10-31 AAL      45.534161  1.011062  3.994389  3.849110  3.921750   
           AAPL     39.767765 -0.906642  3.689797  3.596048  3.642922   
           ABBV     68.121437  0.375557  4.298592  4.205859  4.252226   
           ABT      48.733879 -1.040044  3.944558  3.897414  3.920986   
           ACN     129.900772 -0.986514  4.885869  4.806508  4.846188   
...                       ...       ...       ...       ...       ...   
2023-09-30 WMT      53.783062 -0.196381  4.023309  3.988367  4.005838   
           XOM     114.292953  0.601335  4.775304  4.695106  4.735205   
           MRNA     98.120003 -0.529511  4.788149  4.582514  4.685332   
           UBER     44.270000 -0.746098  3.917801  3.806654  3.862227   
           ABNB    132.279999 -1.006939  5.024801  4.857047  4.940924   

                     garman      macd        rsi  return_1m  return_2m  \
date       ticker                                                        
2017-10-31 AAL    -0.000363 -0.018698  41.051749  -0.014108   0.022981   
           AAPL   -0.000998 -0.039275  69.196704   0.096808   0.015249   
           ABBV   -0.031904  0.473814  55.247824   0.022728   0.098590   
           ABT    -0.004755  0.276134  53.844930   0.021276   0.034308   
           ACN    -0.003628  0.352343  69.365378   0.064180   0.048455   
...                     ...       ...        ...        ...        ...   
2023-09-30 WMT    -0.000011  0.399459  54.722559  -0.000676   0.010014   
           XOM     0.000026  1.400623  59.440178   0.046947   0.046139   
           MRNA    0.000146 -0.376899  38.747314  -0.132219  -0.086803   
           UBER    0.000441 -0.133973  45.005268  -0.062672  -0.053920   
           ABNB    0.000213 -0.037854  44.494127   0.005549  -0.067704   

                   return_3m  return_6m  return_9m  return_12m  
date       ticker                                               
2017-10-31 AAL     -0.023860   0.016495   0.007008    0.012702  
           AAPL     0.044955   0.028875   0.038941    0.035228  
           ABBV     0.091379   0.056495   0.047273    0.044026  
           ABT      0.034801   0.038672   0.031320    0.029294  
           ACN      0.037203   0.028692   0.027398    0.018728  
...                      ...        ...        ...         ...  
2023-09-30 WMT      0.012354   0.017574   0.016553    0.020256  
           XOM      0.030496   0.012838   0.008747    0.027037  
           MRNA    -0.068763  -0.071952  -0.064976   -0.015431  
           UBER     0.008422   0.057244   0.066838    0.043691  
           ABNB     0.010603   0.010289   0.049124    0.019401  

[10341 rows x 14 columns]